removes slt, lpis, and cat surg codes 

Remove suspects and hypertensives from negatives 

require two separate glaucoma diagnoses for negatives

require 120 days of follow-up at least for negatives 

In [1]:
import pandas as pd
import re 
import sqlite3
import numpy as np
import ast
import sklearn 
import math 

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [2]:
conn = sqlite3.connect('../../../dbstridefull.db')
conn.text_factory = str
cur = conn.cursor()

# Get Notes / Unstructured Data for Cohort 

In [3]:
dfsurgnotes=pd.read_sql_query('''select pat_deid, note_deid, substr(note_date, 0, 10) as note_dateonly, effective_dept_id, csn_deid, note from notes 
where note_desc in ("Progress Notes", "Clinic Visit") 
and length(note) > 100 
and  pat_deid in (select distinct pat_deid
from procedure 
where  code IN (
                '66150',
                '66155',
                '66160',
                '66165',
                '66170',
                '66172',
                '66174',
                '66175',
                '66179',
                '66180',
                '66183',
                '66184',
                '66185',
                '67250',
                '67255',
                '0191T',
                '0376T',
                '0474T',
                '0253T',
                '0449T',
                '0450T',
                '0192T',
                '65820',
                '65850',
                '66700',
                '66710',
                '66711',
                '66720',
                '66740',
                '66625',
                '66540'
            )
			) 
order by pat_deid''', conn)
dfsurgnotes.columns = map(str.lower, dfsurgnotes.columns)

In [ ]:
dfsurgnotes.head() #note that this is already sorted by note_date
#cell output cleared to protect phi 

In [6]:
len(dfsurgnotes)

48682

In [7]:
dfsurgnotes["note_dateonly"]=pd.to_datetime(dfsurgnotes["note_dateonly"])
#but we'll sort again just to be absolutely certain: 
dfsurgnotes.sort_values(by=["pat_deid", "note_dateonly"], ascending=True, inplace=True) 
dfsurgnotes['notecounter'] = dfsurgnotes.groupby(["pat_deid"]).cumcount()+1
#keep up to first three visit notes only 
dfsurgnotes=dfsurgnotes[dfsurgnotes["notecounter"]<=3]
dfsurgnotes["firstnotedate"]=np.where(dfsurgnotes['notecounter']==1, dfsurgnotes["note_dateonly"], None)
dfsurgnotes["firstnotedate"]=dfsurgnotes["firstnotedate"].ffill()
dfsurgnotes["firstnotedate"]=pd.to_datetime(dfsurgnotes["firstnotedate"])
dfsurgnotes["daysfromfirstnote"] = dfsurgnotes["note_dateonly"]-dfsurgnotes["firstnotedate"]
dfsurgnotes=dfsurgnotes[dfsurgnotes["daysfromfirstnote"]<=pd.Timedelta(days=120)]

In [9]:
#dfsurgnotes.head()
len(dfsurgnotes)
#dfsurgnotes["pat_deid"].value_counts()

3449

In [10]:
#now exclude the patients who got surgery before 120 days since we need this as a lookback period 
dfsurgproc=pd.read_sql_query('''select distinct pat_deid, proc_date
from procedure 
where  code IN (
                '66150',
                '66155',
                '66160',
                '66165',
                '66170',
                '66172',
                '66174',
                '66175',
                '66179',
                '66180',
                '66183',
                '66184',
                '66185',
                '67250',
                '67255',
                '0191T',
                '0376T',
                '0474T',
                '0253T',
                '0449T',
                '0450T',
                '0192T',
                '65820',
                '65850',
                '66700',
                '66710',
                '66711',
                '66720',
                '66740',
                '66625',
                '66540'
            )
order by pat_deid''', conn)
dfsurgproc.columns = map(str.lower, dfsurgproc.columns)

In [11]:
dfsurgproc["proc_date"]=pd.to_datetime(dfsurgproc["proc_date"])
dfsurgproc.sort_values(by=["pat_deid", "proc_date"], ascending=True, inplace=True) 
dfsurgproc['surgcounter'] = dfsurgproc.groupby(["pat_deid"]).cumcount()+1
#keep first surg only 
dfsurgproc=dfsurgproc[dfsurgproc["surgcounter"]<=1]

In [ ]:
#output cleared to protect phi 
dfsurgproc.head()
dfsurgproc["pat_deid"].value_counts()

In [14]:
dfsurg=pd.merge(dfsurgnotes, dfsurgproc, left_on="pat_deid", right_on="pat_deid", how="left")
dfsurg["daystosurg"]=dfsurg["proc_date"]-dfsurg["firstnotedate"]
#dfsurg[dfsurg["daystosurg"]<pd.Timedelta(days=120)]["pat_deid"].value_counts() #how many throwing away?
dfsurg=dfsurg[dfsurg["daystosurg"]>=pd.Timedelta(days=120)]

In [16]:
#dfsurg.head()
len(dfsurg)
len(dfsurg["pat_deid"].value_counts())

1820

748

#this is an aside, to export the notes before concatenating in order to run Luisa's preprocessing 
#eventually not used in this project
dfsurg[['pat_deid', 'note_deid', 'note']].to_csv('predictglaucomasurgery/surgcohortnotes.csv', index=False)

In [17]:
#concatenate notes# 
dfsurgconcat=dfsurg.groupby(['pat_deid'])['note'].apply(lambda x: ','.join(x)).reset_index()

In [19]:
#dfsurgconcat.head()
#dfsurgconcat["note"][0]
len(dfsurgconcat)

748

In [16]:
#dfsurgconcat.to_csv("predictglaucomasurgyesnolaser.csv")

In [20]:
#now we have to do the same and compare to a cohort of glaucoma patients who didn't get surgery 
#select patients with any glaucoma diagnosis - who aren't part of the above list 
dfnosurgnotes=pd.read_sql_query('''select pat_deid, note_deid, substr(note_date, 0, 10) as note_dateonly, effective_dept_id, csn_deid, note from notes 
where note_desc = "Progress Notes" 
and length(note) > 100 
and pat_deid in (select pat_deid from diagnoses 
where (replace(icd10_list,'.','') like 'H40%'
or replace(icd10_list,'.','') like 'H42%'
or replace(icd10_list,'.','') like 'Q150%') 
and replace(icd10_list,'.','') not like 'H400%'
group by pat_deid 
having count(pat_deid) >=2)
and pat_deid not in (select distinct pat_deid
from procedure 
where  code IN (
                '66150',
                '66155',
                '66160',
                '66165',
                '66170',
                '66172',
                '66174',
                '66175',
                '66179',
                '66180',
                '66183',
                '66184',
                '66185',
                '67250',
                '67255',
                '0191T',
                '0376T',
                '0474T',
                '0253T',
                '0449T',
                '0450T',
                '0192T',
                '65820',
                '65850',
                '66700',
                '66710',
                '66711',
                '66720',
                '66740',
                '66625',
                '66540'
            ))
''', conn)
dfnosurgnotes.columns = map(str.lower, dfnosurgnotes.columns)

#removing 'H400' removes all suspects, ocular hypertensives, borderline findings, steroid response, and narrow angle diagnoses, e.g. people who don't really have true glaucoma  

 H40.0 Glaucoma suspect
 H40.00 Preglaucoma, unspecified
 H40.001 …… right eye
 H40.002 …… left eye
 H40.003 …… bilateral
 H40.009 …… unspecified eye
 H40.01 Open angle with borderline findings, low risk
 H40.011 …… right eye
 H40.012 …… left eye
 H40.013 …… bilateral
 H40.019 …… unspecified eye
 H40.02 Open angle with borderline findings, high risk
 H40.021 …… right eye
 H40.022 …… left eye
 H40.023 …… bilateral
 H40.029 …… unspecified eye
 H40.03 Anatomical narrow angle
 H40.031 …… right eye
 H40.032 …… left eye
 H40.033 …… bilateral
 H40.039 …… unspecified eye
 H40.04 Steroid responder
 H40.041 …… right eye
 H40.042 …… left eye
 H40.043 …… bilateral
 H40.049 …… unspecified eye
 H40.05 Ocular hypertension
 H40.051 …… right eye
 H40.052 …… left eye
 H40.053 …… bilateral
 H40.059 …… unspecified eye
 H40.06 Primary angle closure without glaucoma damage
 H40.061 …… right eye
 H40.062 …… left eye
 H40.063 …… bilateral
 H40.069 …… unspecified eye

In [21]:
dfnosurgnotes["note_dateonly"]=pd.to_datetime(dfnosurgnotes["note_dateonly"])
#but we'll sort again just to be absolutely certain: 
dfnosurgnotes.sort_values(by=["pat_deid", "note_dateonly"], ascending=True, inplace=True) 
dfnosurgnotes['notecounter'] = dfnosurgnotes.groupby(["pat_deid"]).cumcount()+1
dfnosurgnotes["firstnotedate"]=np.where(dfnosurgnotes['notecounter']==1, dfnosurgnotes["note_dateonly"], None)
dfnosurgnotes["firstnotedate"]=dfnosurgnotes["firstnotedate"].ffill()
dfnosurgnotes["firstnotedate"]=pd.to_datetime(dfnosurgnotes["firstnotedate"])
dfnosurgnotes["daysfromfirstnote"] = dfnosurgnotes["note_dateonly"]-dfnosurgnotes["firstnotedate"]

#figure out who had more than 120 days follow-up 
dfnosurgfollowup=dfnosurgnotes[dfnosurgnotes["daysfromfirstnote"]>=pd.Timedelta(days=120)]


In [22]:
len(set(dfnosurgfollowup["pat_deid"])) #3764 patients with greater than 120 days of follow-up 
followuplist=list(set(dfnosurgfollowup["pat_deid"]))

3764

In [23]:
#keep up to first three visit notes only and if they werewithin the first 120 days 
dfnosurgnotes=dfnosurgnotes[dfnosurgnotes["notecounter"]<=3]
dfnosurgnotes=dfnosurgnotes[dfnosurgnotes["daysfromfirstnote"]<=pd.Timedelta(days=120)]

In [24]:
#and then only keep them if they had >120 days of follow-up 
dfnosurgnotes=dfnosurgnotes[dfnosurgnotes["pat_deid"].isin(followuplist)]

In [ ]:
dfnosurgnotes.head()
#cell output cleared for PHI protection 

#this is an aside, to export the notes before concatenating in order to run Luisa's preprocessing 
#not needed for this project 
dfnosurgnotes[['pat_deid', 'note_deid', 'note']].to_csv('predictglaucomasurgery/nosurgcohortnotes.csv', index=False)

In [26]:
dfnosurgconcat=dfnosurgnotes.groupby(['pat_deid'])['note'].apply(lambda x: ','.join(x)).reset_index()

In [ ]:
dfnosurgconcat.head()
#cell output cleared to protect phi 

In [28]:
len(dfnosurgconcat) 

3764

In [153]:
#dfnosurgconcat.to_csv("predictglaucomasurgerynofilterednegs.csv")

# Get Structured Data for Cohort  - Surgical

In [44]:
dfpt=pd.read_sql_query('''select pat_deid, birth_date, gender, race, ethnicity from patients 
where pat_deid in (select distinct pat_deid
from procedure 
where  code IN (
                '66150',
                '66155',
                '66160',
                '66165',
                '66170',
                '66172',
                '66174',
                '66175',
                '66179',
                '66180',
                '66183',
                '66184',
                '66185',
                '67250',
                '67255',
                '0191T',
                '0376T',
                '0474T',
                '0253T',
                '0449T',
                '0450T',
                '0192T',
                '65820',
                '65850',
                '66700',
                '66710',
                '66711',
                '66720',
                '66740',
                '66625',
                '66540'
            )
			) 
order by pat_deid''', conn)
dfpt.columns = map(str.lower, dfpt.columns)

In [45]:
dfpt["birth_date"]=pd.to_datetime(dfpt["birth_date"])
from datetime import timedelta, date
def fix_date(x):
    if x.year >=2010:
        year = x.year - 100
    else:
        year = x.year
    return date(year,x.month,x.day)

dfpt['birth_date'] = dfpt['birth_date'].apply(fix_date)

In [46]:
#need diagnoses, medications, and optionally eye exam fields, but only from the first 3 mos 
dfsurgfirstdate=dfsurgnotes[["pat_deid", "firstnotedate"]]
len(dfsurgfirstdate)
dfsurgfirstdate=dfsurgfirstdate.drop_duplicates() 
len(dfsurgfirstdate)

3449

1298

In [47]:
dfpt=pd.merge(dfpt,dfsurgfirstdate, left_on="pat_deid", right_on="pat_deid", how="right")

In [54]:
dfpt["age"]=pd.to_datetime(dfpt["firstnotedate"]).dt.year-pd.to_datetime(dfpt["birth_date"]).dt.year

In [ ]:
del dfpt["firstnotedate"]
del dfpt["birth_date"]
dfpt.head()

In [56]:
#diagnoses 
dfdx=pd.read_sql_query('''select pat_deid, start_date, icd10_list from diagnoses where pat_deid in (select distinct pat_deid
from procedure 
where  code IN (
                '66150',
                '66155',
                '66160',
                '66165',
                '66170',
                '66172',
                '66174',
                '66175',
                '66179',
                '66180',
                '66183',
                '66184',
                '66185',
                '67250',
                '67255',
                '0191T',
                '0376T',
                '0474T',
                '0253T',
                '0449T',
                '0450T',
                '0192T',
                '65820',
                '65850',
                '66700',
                '66710',
                '66711',
                '66720',
                '66740',
                '66625',
                '66540'
            )
			) 
order by pat_deid''', conn)
dfdx.columns = map(str.lower, dfdx.columns)

In [57]:
dfdx["dx_date"]=pd.to_datetime(dfdx["start_date"])

In [58]:
del dfdx["start_date"]

In [60]:
len(dfdx)
#dfdx.head()

333475

In [61]:
dfdx=pd.merge(dfdx,dfsurgfirstdate, left_on="pat_deid", right_on="pat_deid", how="right")
dfdx.sort_values(by=["pat_deid", "dx_date"], ascending=True, inplace=True)
dfdx=dfdx[(dfdx["dx_date"]-dfdx["firstnotedate"])<=pd.Timedelta(days=120)]

In [ ]:
dfdx.head()

In [63]:
dfdx["pivotvalue"]=1

In [65]:
dfdxwide=dfdx.pivot_table(values="pivotvalue", index='pat_deid', columns='icd10_list', fill_value=0)
dfdxwide.columns = ['icd_'+col for col in dfdxwide.columns.values]
dfdxwide.reset_index(inplace=True)
#dfdxwide.head()

In [66]:
len(dfdxwide)

1265

In [67]:
#meds 
dfmeds=pd.read_sql_query('''select pat_deid, order_time, medication_id from meds where pat_deid in (select distinct pat_deid
from procedure 
where  code IN (
                '66150',
                '66155',
                '66160',
                '66165',
                '66170',
                '66172',
                '66174',
                '66175',
                '66179',
                '66180',
                '66183',
                '66184',
                '66185',
                '67250',
                '67255',
                '0191T',
                '0376T',
                '0474T',
                '0253T',
                '0449T',
                '0450T',
                '0192T',
                '65820',
                '65850',
                '66700',
                '66710',
                '66711',
                '66720',
                '66740',
                '66625',
                '66540'
            )
			) 
order by pat_deid''', conn)
dfmeds.columns = map(str.lower, dfmeds.columns)

In [68]:
dfmeds["rx_date"]=pd.to_datetime(dfmeds["order_time"])

In [69]:
del dfmeds["order_time"]

In [70]:
len(dfmeds)
#dfmeds.head()

85394

,pat_deid,medication_id,rx_date
0,1861,22455,2016-11-21
1,1861,10814,2015-07-15
2,1861,26482,2015-11-20
3,1861,213228,2015-07-15
4,1861,84953,2016-11-21


In [71]:
dfmeds=pd.merge(dfmeds,dfsurgfirstdate, left_on="pat_deid", right_on="pat_deid", how="right")
dfmeds.sort_values(by=["pat_deid", "rx_date"], ascending=True, inplace=True)

In [72]:
#keep meds before 120 days after first note 
dfmeds=dfmeds[(dfmeds["rx_date"]<=(dfmeds["firstnotedate"])+pd.Timedelta(days=120))]

In [75]:
#dfmeds.head()
len(dfmeds)

24839

In [77]:
dfmeds["pivotvalue"]=1
dfmeds["medication_id"]=dfmeds["medication_id"].astype(int)
dfmedswide=dfmeds.pivot_table(values="pivotvalue", index='pat_deid', columns='medication_id', fill_value=0)
dfmedswide.columns = ['med_'+str(col) for col in dfmedswide.columns.values]
dfmedswide.reset_index(inplace=True)
#dfmedswide.head()

In [78]:
len(dfmedswide)

1269

In [82]:
#eye exam fields - VA, IOP - stuff we would have on everyone 
dfexam=pd.read_sql_query('''select pat_deid, DATE_OF_SERVICE, 
vaoddistcc, vaoddistsc, vaoddistccph, vaoddistscph, vaosdistcc, vaosdistsc, vaosdistccph, vaosdistscph, tod, tos, tmethod
from examfield, examparsed 
where examfield.smartformid = examparsed.smartformid 
and not (vaoddistcc is null and vaoddistsc is null and vaosdistcc is null and vaosdistsc is null)
and pat_deid in (select distinct pat_deid
from procedure 
where  code IN (
                '66150',
                '66155',
                '66160',
                '66165',
                '66170',
                '66172',
                '66174',
                '66175',
                '66179',
                '66180',
                '66183',
                '66184',
                '66185',
                '67250',
                '67255',
                '0191T',
                '0376T',
                '0474T',
                '0253T',
                '0449T',
                '0450T',
                '0192T',
                '65820',
                '65850',
                '66700',
                '66710',
                '66711',
                '66720',
                '66740',
                '66625',
                '66540'
            )
			) 
            order by pat_deid 
''', conn) 
dfexam.columns = map(str.lower, dfexam.columns)

In [83]:
dfexam["exam_date"]=pd.to_datetime(dfexam["date_of_service"])
del dfexam["date_of_service"]
len(dfexam)
#dfexam.head()

51979

In [84]:
dfexam=pd.merge(dfexam,dfsurgfirstdate, left_on="pat_deid", right_on="pat_deid", how="right")
dfexam.sort_values(by=["pat_deid", "exam_date"], ascending=True, inplace=True)
dfexam=dfexam[(dfexam["exam_date"]-dfexam["firstnotedate"])<=pd.Timedelta(days=120)]

In [86]:
len(dfexam)
#dfexam.head()

8699

In [87]:
#let's process the IOPs 
#we will get the max IOP in each list (for each date), then take the maxiop over all the dates per patient
dft=dfexam[["pat_deid", "exam_date", "tod", "tos"]]
dft=dft[(dft["tod"]!="null") | (dft["tos"]!="null")]

In [88]:
def getmaxt(stringlist): 
    try: 
        tlist=ast.literal_eval(stringlist)
    except: 
        return np.nan 
    numlist=[] 
    for item in tlist: 
        try: 
            itemint=int(item)
            numlist.append(itemint)
        except: continue  
    try: 
        maxt=max(numlist)
    except:
        maxt=np.nan
    return maxt 

In [ ]:
dft["todmax"]=dft["tod"].apply(getmaxt)
dft["tosmax"]=dft["tos"].apply(getmaxt)
dft.head()
#cell output cleared for PHI 

In [ ]:
dfmaxt=pd.concat([dft[["pat_deid", "todmax"]].groupby(["pat_deid"]).max(),dft[["pat_deid", "tosmax"]].groupby(["pat_deid"]).max()], axis=1). reset_index()
dfmaxt.head()
#cell output cleared for PHI 

In [91]:
#now process the VA's 
dfva=dfexam[["pat_deid", "exam_date", "vaoddistsc", "vaoddistcc", "vaoddistscph", "vaoddistccph","vaosdistsc", "vaosdistcc", "vaosdistscph", "vaosdistccph"]]

In [93]:
def logmarconversion(va): 
    #takes a string input in the form of "20/20", or "cf" "hm" etc and spits out a logmar 
    if len(re.findall('(?i)cf', va))>0: 
        logmarva=-np.log10(.0025)
    elif len(re.findall('(?i)hm', va))>0: 
        logmarva=-np.log10(.002)
    elif len(re.findall('(?i)nlp', va))>0:
        logmarva=-np.log10(0.0013)
    elif len(re.findall('(?i)lp', va))>0: 
        logmarva=-np.log10(0.0016)
    elif len(re.findall('(?i)20/1600', va))>0: 
        logmarva=-np.log10(20/1600)
    elif len(re.findall('(?i)20/1250', va))>0: 
        logmarva=-np.log10(20/1250)
    elif len(re.findall('(?i)20/1000', va))>0: 
        logmarva=-np.log10(20/1000)
    elif len(re.findall('(?i)20/800', va))>0: 
        logmarva=-np.log10(20/800)
    elif len(re.findall('(?i)20/650', va))>0: 
        logmarva=-np.log10(20/650)
    elif len(re.findall('(?i)20/500', va))>0: 
        logmarva=-np.log10(20/500)
    elif len(re.findall('(?i)20/400', va))>0: 
        logmarva=-np.log10(20/400)
    elif len(re.findall('(?i)20/350', va))>0: 
        logmarva=-np.log10(20/350)
    elif len(re.findall('(?i)20/300', va))>0: 
        logmarva=-np.log10(20/300)
    elif len(re.findall('(?i)20/250', va))>0: 
        logmarva=-np.log10(20/250)
    elif len(re.findall('(?i)20/225', va))>0: 
        logmarva=-np.log10(20/225)
    elif len(re.findall('(?i)20/200', va))>0: 
        logmarva=-np.log10(20/200)
    elif len(re.findall('(?i)20/160', va))>0: 
        logmarva=-np.log10(20/160)
    elif len(re.findall('(?i)20/150', va))>0: 
        logmarva=-np.log10(20/150)
    elif len(re.findall('(?i)20/125', va))>0: 
        logmarva=-np.log10(20/125)
    elif len(re.findall('(?i)20/120', va))>0: 
        logmarva=-np.log10(20/120)
    elif len(re.findall('(?i)20/100', va))>0: 
        logmarva=-np.log10(20/100)
    elif len(re.findall('(?i)20/80', va))>0: 
        logmarva=-np.log10(20/80)
    elif len(re.findall('(?i)20/70', va))>0: 
        logmarva=-np.log10(20/70)
    elif len(re.findall('(?i)20/63', va))>0: 
        logmarva=-np.log10(20/63)
    elif len(re.findall('(?i)20/60', va))>0: 
        logmarva=-np.log10(20/60)
    elif len(re.findall('(?i)20/50', va))>0: 
        logmarva=-np.log10(20/50)
    elif len(re.findall('(?i)20/40', va))>0: 
        logmarva=-np.log10(20/40)
    elif len(re.findall('(?i)20/32', va))>0: 
        logmarva=-np.log10(20/32)
    elif len(re.findall('(?i)20/30', va))>0: 
        logmarva=-np.log10(20/30)
    elif len(re.findall('(?i)20/25', va))>0: 
        logmarva=-np.log10(20/25)
    elif len(re.findall('(?i)20/20', va))>0: 
        logmarva=-np.log10(20/20)
    elif len(re.findall('(?i)20/16', va))>0: 
        logmarva=-np.log10(20/16)
    elif len(re.findall('(?i)20/15', va))>0: 
        logmarva=-np.log10(20/15)
    elif len(re.findall('(?i)20/10', va))>0: 
        logmarva=-np.log10(20/10)
        
    else: logmarva=np.nan 
    return logmarva 

#now write a function which will take several va inputs and output the bcva logmar 
def bcvalogmar(vadistsc, vadistcc, vadistscph, vadistccph): 
    valist=list(filter(None.__ne__, [vadistsc, vadistcc, vadistscph, vadistccph])) #filter's out whichever are None
    logmarlist=[] 
    for va in valist: 
        if np.isnan(logmarconversion(va)) == False: 
            logmarlist.append(logmarconversion(va)) 
    try: bcvalogmar=min(logmarlist)
    except ValueError: bcvalogmar=np.nan #if no va's were recorded and all None input then this is an empty list with no minum
    return bcvalogmar 


In [94]:
dfva["bcvalogmarod"]=dfva[["vaoddistsc", "vaoddistcc", "vaoddistscph", "vaoddistccph"]].apply(lambda x: bcvalogmar(*x), axis=1)
dfva["bcvalogmaros"]=dfva[["vaosdistsc", "vaosdistcc", "vaosdistscph", "vaosdistccph"]].apply(lambda x: bcvalogmar(*x), axis=1)

C:\Users\Sophia\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
C:\Users\Sophia\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [ ]:
dfva=dfva[["pat_deid", "bcvalogmarod", "bcvalogmaros"]]
dfbcva=pd.concat([dfva[["pat_deid", "bcvalogmarod"]].groupby(["pat_deid"]).min(),dfva[["pat_deid", "bcvalogmaros"]].groupby(["pat_deid"]).min()], axis=1). reset_index()
dfbcva.head()
#cell output cleared for PHI 

In [ ]:
dfsurgexamfinal=pd.merge(dfbcva, dfmaxt, on="pat_deid", how="outer")
dfsurgexamfinal.head()
#cell output cleared for PHI 

In [97]:
len(dfsurgexamfinal)

1261

In [ ]:
#now we do a giant merge of the data to save it 
dfsurgexamfinal 
dfmedswide
dfdxwide
dfpt
#cell output cleared for PHI 

In [99]:
dfsurgstruct=pd.merge(pd.merge(pd.merge(dfpt, dfdxwide, on="pat_deid", how="outer"), 
         dfmedswide, on='pat_deid', how="outer").fillna(0), 
        dfsurgexamfinal, on='pat_deid', how="outer")

In [100]:
dfsurgall=pd.merge(dfsurgconcat,dfsurgstruct, on='pat_deid', how='left')
#dfsurgall.to_csv("predictglaucomasurgeryyesallwolaser.csv")

# Get Structured Data for Cohort  - Non Surgical

In [101]:
dfpt=pd.read_sql_query('''select pat_deid, birth_date, gender, race, ethnicity from patients
where pat_deid in (select pat_deid from diagnoses 
where (replace(icd10_list,'.','') like 'H40%'
or replace(icd10_list,'.','') like 'H42%'
or replace(icd10_list,'.','') like 'Q150%') 
and replace(icd10_list,'.','') not like 'H400%'
group by pat_deid 
having count(pat_deid) >=2)
and pat_deid not in (select distinct pat_deid
from procedure 
where  code IN (
                '66150',
                '66155',
                '66160',
                '66165',
                '66170',
                '66172',
                '66174',
                '66175',
                '66179',
                '66180',
                '66183',
                '66184',
                '66185',
                '67250',
                '67255',
                '0191T',
                '0376T',
                '0474T',
                '0253T',
                '0449T',
                '0450T',
                '0192T',
                '65820',
                '65850',
                '66700',
                '66710',
                '66711',
                '66720',
                '66740',
                '66625',
                '66540'
            )) order by pat_deid 
''', conn)
dfpt.columns = map(str.lower, dfpt.columns)

In [ ]:
dfpt["birth_date"]=pd.to_datetime(dfpt["birth_date"])
from datetime import timedelta, date
def fix_date(x):
    if x.year >=2010:
        year = x.year - 100
    else:
        year = x.year
    return date(year,x.month,x.day)

dfpt['birth_date'] = dfpt['birth_date'].apply(fix_date)

dfpt.head()
#cell output cleared

In [103]:
#need diagnoses, medications, and optionally eye exam fields, but only from the first 3 mos 
dfnosurgfirstdate=dfnosurgnotes[["pat_deid", "firstnotedate"]]
len(dfnosurgfirstdate)
dfnosurgfirstdate=dfnosurgfirstdate.drop_duplicates() 
len(dfnosurgfirstdate)

8744

3764

In [ ]:
dfpt=pd.merge(dfpt,dfnosurgfirstdate, left_on="pat_deid", right_on="pat_deid", how="right")

dfpt["age"]=pd.to_datetime(dfpt["firstnotedate"]).dt.year-pd.to_datetime(dfpt["birth_date"]).dt.year
del dfpt["firstnotedate"]
del dfpt["birth_date"]
dfpt.head()

#cell output cleared 

In [105]:
#diagnoses 
dfdx=pd.read_sql_query('''select pat_deid, start_date, icd10_list from diagnoses 
where pat_deid in (select pat_deid from diagnoses 
where (replace(icd10_list,'.','') like 'H40%'
or replace(icd10_list,'.','') like 'H42%'
or replace(icd10_list,'.','') like 'Q150%') 
and replace(icd10_list,'.','') not like 'H400%'
group by pat_deid 
having count(pat_deid) >=2)
and pat_deid not in (select distinct pat_deid
from procedure 
where  code IN (
                '66150',
                '66155',
                '66160',
                '66165',
                '66170',
                '66172',
                '66174',
                '66175',
                '66179',
                '66180',
                '66183',
                '66184',
                '66185',
                '67250',
                '67255',
                '0191T',
                '0376T',
                '0474T',
                '0253T',
                '0449T',
                '0450T',
                '0192T',
                '65820',
                '65850',
                '66700',
                '66710',
                '66711',
                '66720',
                '66740',
                '66625',
                '66540'
            )) order by pat_deid 
''', conn)
dfdx.columns = map(str.lower, dfdx.columns)

In [106]:
dfdx["dx_date"]=pd.to_datetime(dfdx["start_date"])

In [107]:
del dfdx["start_date"]

In [109]:
len(dfdx)
#dfdx.head()

580908

In [110]:
dfdx=pd.merge(dfdx,dfnosurgfirstdate, left_on="pat_deid", right_on="pat_deid", how="right")
dfdx.sort_values(by=["pat_deid", "dx_date"], ascending=True, inplace=True)
dfdx=dfdx[(dfdx["dx_date"]-dfdx["firstnotedate"])<=pd.Timedelta(days=120)]

In [113]:
#dfdx.head()

In [112]:
dfdx["pivotvalue"]=1

In [ ]:
dfdxwide=dfdx.pivot_table(values="pivotvalue", index='pat_deid', columns='icd10_list', fill_value=0)
dfdxwide.columns = ['icd_'+col for col in dfdxwide.columns.values]
dfdxwide.reset_index(inplace=True)
dfdxwide.head()
#cell output cleared

In [115]:
len(dfdxwide)

3744

In [116]:
#meds 
dfmeds=pd.read_sql_query('''select pat_deid, order_time, medication_id from meds 
where pat_deid in (select pat_deid from diagnoses 
where (replace(icd10_list,'.','') like 'H40%'
or replace(icd10_list,'.','') like 'H42%'
or replace(icd10_list,'.','') like 'Q150%') 
and replace(icd10_list,'.','') not like 'H400%'
group by pat_deid 
having count(pat_deid) >=2)
and pat_deid not in (select distinct pat_deid
from procedure 
where  code IN (
                '66150',
                '66155',
                '66160',
                '66165',
                '66170',
                '66172',
                '66174',
                '66175',
                '66179',
                '66180',
                '66183',
                '66184',
                '66185',
                '67250',
                '67255',
                '0191T',
                '0376T',
                '0474T',
                '0253T',
                '0449T',
                '0450T',
                '0192T',
                '65820',
                '65850',
                '66700',
                '66710',
                '66711',
                '66720',
                '66740',
                '66625',
                '66540'
            )) order by pat_deid 
''', conn)
dfmeds.columns = map(str.lower, dfmeds.columns)

In [117]:
dfmeds["rx_date"]=pd.to_datetime(dfmeds["order_time"])

In [118]:
del dfmeds["order_time"]

In [122]:
len(dfmeds)
#dfmeds.head()

70593

In [120]:
dfmeds=pd.merge(dfmeds,dfnosurgfirstdate, left_on="pat_deid", right_on="pat_deid", how="right")
dfmeds.sort_values(by=["pat_deid", "rx_date"], ascending=True, inplace=True)

In [121]:
#keep meds before 120 days after first note 
dfmeds=dfmeds[(dfmeds["rx_date"]<=(dfmeds["firstnotedate"])+pd.Timedelta(days=120))]

In [123]:
#dfmeds.head()
len(dfmeds)

70593

In [ ]:
dfmeds["pivotvalue"]=1
dfmeds["medication_id"]=dfmeds["medication_id"].astype(int)
dfmedswide=dfmeds.pivot_table(values="pivotvalue", index='pat_deid', columns='medication_id', fill_value=0)
dfmedswide.columns = ['med_'+str(col) for col in dfmedswide.columns.values]
dfmedswide.reset_index(inplace=True)
dfmedswide.head()
#cell output cleared

In [125]:
len(dfmedswide)

3633

In [126]:
#eye exam fields - VA, IOP - stuff we would have on everyone 
dfexam=pd.read_sql_query('''select pat_deid, DATE_OF_SERVICE, 
vaoddistcc, vaoddistsc, vaoddistccph, vaoddistscph, vaosdistcc, vaosdistsc, vaosdistccph, vaosdistscph, tod, tos, tmethod
from examfield, examparsed 
where examfield.smartformid = examparsed.smartformid 
and not (vaoddistcc is null and vaoddistsc is null and vaosdistcc is null and vaosdistsc is null)
and pat_deid in (select pat_deid from diagnoses 
where (replace(icd10_list,'.','') like 'H40%'
or replace(icd10_list,'.','') like 'H42%'
or replace(icd10_list,'.','') like 'Q150%') 
and replace(icd10_list,'.','') not like 'H400%'
group by pat_deid 
having count(pat_deid) >=2)
and pat_deid not in (select distinct pat_deid
from procedure 
where  code IN (
                '66150',
                '66155',
                '66160',
                '66165',
                '66170',
                '66172',
                '66174',
                '66175',
                '66179',
                '66180',
                '66183',
                '66184',
                '66185',
                '67250',
                '67255',
                '0191T',
                '0376T',
                '0474T',
                '0253T',
                '0449T',
                '0450T',
                '0192T',
                '65820',
                '65850',
                '66700',
                '66710',
                '66711',
                '66720',
                '66740',
                '66625',
                '66540'
            )) order by pat_deid
''', conn)
dfexam.columns = map(str.lower, dfexam.columns)

In [ ]:
dfexam["exam_date"]=pd.to_datetime(dfexam["date_of_service"])
del dfexam["date_of_service"]
len(dfexam)
dfexam.head()
#cell output cleared

In [128]:
dfexam=pd.merge(dfexam,dfnosurgfirstdate, left_on="pat_deid", right_on="pat_deid", how="right")
dfexam.sort_values(by=["pat_deid", "exam_date"], ascending=True, inplace=True)
dfexam=dfexam[(dfexam["exam_date"]-dfexam["firstnotedate"])<=pd.Timedelta(days=120)]

In [130]:
len(dfexam)
#dfexam.head()

14217

In [131]:
#let's process the IOPs 
#we will get the max IOP in each list (for each date), then take the maxiop over all the dates per patient
dft=dfexam[["pat_deid", "exam_date", "tod", "tos"]]
dft=dft[(dft["tod"]!="null") | (dft["tos"]!="null")]

In [133]:
dft["todmax"]=dft["tod"].apply(getmaxt)
dft["tosmax"]=dft["tos"].apply(getmaxt)
#dft.head()

In [ ]:
dfmaxt=pd.concat([dft[["pat_deid", "todmax"]].groupby(["pat_deid"]).max(),dft[["pat_deid", "tosmax"]].groupby(["pat_deid"]).max()], axis=1). reset_index()
dfmaxt.head()
#cell output cleared

In [135]:
#now process the VA's 
dfva=dfexam[["pat_deid", "exam_date", "vaoddistsc", "vaoddistcc", "vaoddistscph", "vaoddistccph","vaosdistsc", "vaosdistcc", "vaosdistscph", "vaosdistccph"]]

In [137]:
#dfva.head()

In [138]:
dfva["bcvalogmarod"]=dfva[["vaoddistsc", "vaoddistcc", "vaoddistscph", "vaoddistccph"]].apply(lambda x: bcvalogmar(*x), axis=1)
dfva["bcvalogmaros"]=dfva[["vaosdistsc", "vaosdistcc", "vaosdistscph", "vaosdistccph"]].apply(lambda x: bcvalogmar(*x), axis=1)

C:\Users\Sophia\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
C:\Users\Sophia\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [ ]:
dfva=dfva[["pat_deid", "bcvalogmarod", "bcvalogmaros"]]
dfbcva=pd.concat([dfva[["pat_deid", "bcvalogmarod"]].groupby(["pat_deid"]).min(),dfva[["pat_deid", "bcvalogmaros"]].groupby(["pat_deid"]).min()], axis=1). reset_index()
dfbcva.head()
#cell output cleared

In [ ]:
dfnosurgexamfinal=pd.merge(dfbcva, dfmaxt, on="pat_deid", how="outer")
dfnosurgexamfinal.head()
#cell output cleared

In [141]:
len(dfnosurgexamfinal)

3740

In [142]:
dfnosurgstruct=pd.merge(pd.merge(pd.merge(dfpt, dfdxwide, on="pat_deid", how="outer"), 
         dfmedswide, on='pat_deid', how="outer").fillna(0), 
        dfnosurgexamfinal, on='pat_deid', how="outer")
len(dfnosurgstruct)

3764

In [143]:
dfnosurgall=pd.merge(dfnosurgconcat,dfnosurgstruct, on='pat_deid', how='left')
#dfnosurgall.to_csv("predictglaucomasurgerynoallwolaserfiltered.csv")

# Data preprocessing, removing nzv features, standardizing, etc. 

In [152]:
dfsurgall.shape
dfnosurgall.shape

(748, 4245)

(3764, 7677)

In [153]:
dfsurgall["surgery"]=1
dfnosurgall["surgery"]=0
#dfsurgall.head()
#dfnosurgall.head()

In [154]:
dfcombined=pd.concat([dfsurgall, dfnosurgall], axis=0, join='inner')
dfcombined.shape

(4512, 3325)

From here can save the nonstandardized version of the dataset with all categorical variables before near zero variance filtration - for use in determining glaucoma severity codes ? Except don't save the notes

In [155]:
dfcombined.drop('note', axis=1).to_csv(
    "C:/Users/Sophia/Documents/ResearchPHI/STRIDE_FULL/predictglaucomasurgery/cleanednolaserfilterednegs/predictglaucomasurgerystructnonstandardallcodes.csv", index=None)

In [150]:
#normalize age
dfcombined["agestandard"]=(dfcombined["age"]-dfcombined["age"].mean())/dfcombined["age"].std()
#check and make sure it worked 
dfcombined["agestandard"].mean()
dfcombined["agestandard"].std()

-6.887221289729195e-17

1.0000000000000053

In [ ]:
dfcombined.head()

In [198]:
from sklearn.feature_selection import VarianceThreshold
selector=VarianceThreshold(.99 * (1 - .99))

In [199]:
selector.fit_transform(np.array(dfcombined.loc[:, 'icd_A15.0':'med_541454'])).shape

(4512, 342)

In [200]:
def variance_threshold_selector(data, threshold=0.5):
    selector = VarianceThreshold(threshold)
    selector.fit(data)
    return data[data.columns[selector.get_support(indices=True)]]

In [201]:
dfombinednzvfiltered=variance_threshold_selector(dfcombined.loc[:, 'icd_A15.0':'med_541454'], .99 * (1 - .99))

In [202]:
#normalize eye exam measures 
dfexamstandard=(dfcombined.loc[:,'bcvalogmarod':'tosmax']-dfcombined.loc[:,'bcvalogmarod':'tosmax'].mean())/dfcombined.loc[:,'bcvalogmarod':'tosmax'].std()
#check and make sure it worked 
dfexamstandard.mean()
dfexamstandard.std()

bcvalogmarod    2.914191e-15
bcvalogmaros    2.510221e-15
todmax         -7.005277e-17
tosmax          6.509590e-17
dtype: float64

bcvalogmarod    1.0
bcvalogmaros    1.0
todmax          1.0
tosmax          1.0
dtype: float64

In [203]:
import math 
def missingindicator(x): 
    if math.isnan(x): 
        return 1 
    else: 
        return 0 
dfexamstandard["todmissing"]=dfexamstandard["todmax"].apply(missingindicator)
dfexamstandard["tosmissing"]=dfexamstandard["tosmax"].apply(missingindicator)
dfexamstandard.head()

,bcvalogmarod,bcvalogmaros,todmax,tosmax,todmissing,tosmissing
0,-0.294232,-0.323032,NaN,NaN,1,1
1,-0.531139,-0.548348,NaN,NaN,1,1
2,0.200831,-0.548348,1.106909,0.430014,0,0
3,-0.400760,-0.163168,NaN,NaN,1,1
4,NaN,NaN,NaN,NaN,1,1


In [204]:
#now fill missing values, equivalent to mean imputation 
dfexamstandard=dfexamstandard.fillna(0)

In [205]:
#time to one-hot encode the demographics 
dfdemdummies=pd.get_dummies(dfcombined.loc[:,'gender':'ethnicity'])

In [206]:
dfcombinedprocessed=pd.concat([dfcombined[['pat_deid', 'note','agestandard']], dfdemdummies, dfexamstandard, dfombinednzvfiltered, dfcombined["surgery"]], axis=1)

In [ ]:
dfcombinedprocessed.head()

In [208]:
dfcombinedprocessed.to_csv('predictglaucomasurgerycombinedprocessedwolaserfiltered.csv')

In [209]:
dfcombinedprocessed.shape

(4512, 364)

In [210]:
dfcombinedprocessed["surgery"].value_counts()

0    3764
1     748
Name: surgery, dtype: int64

In [156]:
conn.close()

In [212]:
748/4512

0.16578014184397163